In [40]:
from selenium.common import NoSuchElementException, TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
import time
import chromedriver_autoinstaller
"""
from flask import Flask, request

app = Flask(__name__)

my_place_id = ""

@app.route("/get_place_details", methods=["POST"])
def get_place_details():
    global my_place_id
    data = request.get_json()
    my_place_id = data["place_id"]
    return "Received place id: " + my_place_id

if __name__ == "__main__":
    app.run(port=5000)
"""
#my_place_id = "ChIJj61dQgK6j4AR4GeTYWZsKWw"
#google_url = "https://www.google.com/maps/place/?q=place_id:"+my_place_id
google_url = "https://www.google.com/maps/place/Le+Splendid+-+Salle+de+concert/@50.6293963,3.082782,15z/data=!4m14!1m7!3m6!1s0x47e706d27b324289:0x6ef9f2bbac9fe81d!2sAu+Bureau+Beauvais!8m2!3d49.430828!4d2.084429!16s%2Fg%2F1wl4pxy7!3m5!1s0x47c2d5e41be86ae7:0xc555806cb1a4c7b5!8m2!3d50.6233991!4d3.0906942!16s%2Fg%2F1tg5yssj?entry=ttu"

chromedriver_autoinstaller.install()

driver = webdriver.Chrome()

driver.set_window_size(1024, 600)
driver.maximize_window()

driver.get(google_url)

names = []
rating = []
dates = []
comments = []

try:
    # Wait for consent page to load
    accept_all_button = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, "[jsname='b3VHJd']"))
    )
    # Accept cookies because they are good
    accept_all_button.click()
except:
    print("An error occurred when trying to accept Google's terms of conditions.")

# get numbers of comments
total_reviews = 1
try:
    # Wait for consent page to load
    reviews_info = WebDriverWait(driver, 10).until(
        # By css_selector (1 or more classes) is more robust choice because by class_name can only handle 1 class
        EC.presence_of_element_located((By.CSS_SELECTOR, ".F7nice"))
    )
    # get all the spans
    span_elements = reviews_info.find_elements(By.TAG_NAME, 'span')
    print(len(span_elements))
    total_reviews = int(span_elements[8].text[1:-1].replace(",", ""))
    print(total_reviews)
    if total_reviews > 500:
        total_reviews = 500
    print(total_reviews)
except:
    print("Couldn't get number of reviews.")

# get to reviews
try:
    reviews_button = WebDriverWait(driver, 10).until(
        # EC.presence_of_element_located((By.CSS_SELECTOR, "[data-tab-index='1']"))
        # EC.presence_of_element_located((By.CSS_SELECTOR, "[aria-label*='Review']"))
        EC.presence_of_element_located((By.CSS_SELECTOR, "[jslog='145620;track:click;']"))
    )
    reviews_button.click()
except:
    print("An error occurred when trying to get to reviews.")

# scroll until the end
try:
    scrollable_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".m6QErb.DxyBCb.kA9KIf.dS8AEf"))
    )
    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
    # get number of currently loaded comments
    n_comment_divs = len(driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium "))
    # limits to 500 reviews
    while n_comment_divs < total_reviews:
        # scroll to the bottom
        driver.execute_script(
            "arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_div
        )
        # wait for the page to load
        time.sleep(1.3)
        # recalculate number of loaded divs
        try:
            # wait for the number of comments to change with a maximum wait time of 10 seconds
            WebDriverWait(driver, 10).until(lambda x: len(x.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium ")) > n_comment_divs)
            # recalculate number of loaded divs
            n_comment_divs = len(driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium "))
        except TimeoutException:
            # if after 10 seconds the number of comments hasn't changed, break
            break
        # calculate new scroll height and compare with the last scroll height
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_div)
        if new_height == last_height:
            break
        last_height = new_height
except:
    print("Couldn't scroll...")

# open the comments fully (see more)
try:
    # gather all see more buttons and click them
    all_see_more_buttons = driver.find_elements(By.CSS_SELECTOR, ".w8nwRe.kyuRq")
    for button in all_see_more_buttons:
        button.click()
except:
    print("Couldn't click all see more buttons.")

# extract text
try:
    comment_divs = driver.find_elements(By.CSS_SELECTOR, ".jftiEf.fontBodyMedium ")
    # there is necessarily a name, rating and a date but not comment
    for comment in comment_divs:
        # get name
        names.append(comment.find_element(By.CLASS_NAME, "d4r55 ").text)
        # get review but on the case of hotels, it might be different disposition
        try:
            rating.append(comment.find_element(By.CLASS_NAME, "kvMYJc").get_attribute("aria-label"))
        except:
            rating.append(comment.find_element(By.CLASS_NAME, "fzvQIb").text)
        # get dates, in case of hotels, might be different
        try:
            dates.append(comment.find_element(By.CLASS_NAME, "rsqaWe").text)
        except:
            date_element = comment.find_element(By.CLASS_NAME, "xRkPPb")
            date_text = driver.execute_script('return arguments[0].firstChild.textContent;', date_element).strip()
            dates.append(date_text)
        # verify that there is comment and add, else add empty string
        text = ""
        try:
            myened_element = comment.find_element(By.CLASS_NAME, "MyEned")
            text = myened_element.find_element(By.CLASS_NAME, "wiI7pd").text
        except NoSuchElementException:
            text = ""
        comments.append(text)
except:
    print("Something happened when trying to parse reviews...")

finally:
    driver.quit()

zipped = list(zip(names, rating, dates, comments))
dictionary = {key: (v1, v2, v3) for key, v1, v2, v3 in zipped}

print(dictionary)

11
1355
500
{'moc1203': ('4 stars', '2 months ago', "It's a small venue.  Some seats are broken and you can't sit on them.  There are no coin lockers."), 'Firas Kiki': ('4 stars', '6 months ago', 'So nice and beautiful music experience with a lot of people. I like it'), 'louwave louwave': ('3 stars', 'a month ago', 'So music'), 'Linda Lee Hopkins': ('5 stars', 'a year ago', 'We were well received and the public was just AWESOME !!!   I had a wonderful time.'), 'Blue Butterfly': ('4 stars', '4 years ago', 'Nice Club for Live music'), 'Erik237 59': ('5 stars', '2 months ago', 'Nice'), 'Frederic Muylle (Fré)': ('4 stars', '3 years ago', 'Perfect venue! Love the old style.'), 'Saidalavi Ct': ('5 stars', '4 years ago', 'Very nice'), 'AMAZON TOP PRODUCTS': ('5 stars', 'a year ago', 'top'), 'pereyi Bayefa': ('5 stars', '3 years ago', 'Perfect'), 'K 8': ('5 stars', '3 years ago', 'Good'), 'PA Piat (ELEMENTS KNIVES)': ('5 stars', '4 years ago', 'Top!!!!'), 'bertrand Gasnier': ('5 stars', '3 yea

In [41]:
import csv

# Exporter le dict vers le fichier csv (OVERWRITE)
import csv
chemin_fichier = 'data.csv'

# Écrire le dictionnaire dans le fichier CSV
with open(chemin_fichier, 'w', newline='', encoding='utf-8') as fichier_csv:
    writer = csv.writer(fichier_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Name', 'grading', 'date', 'Text'])  # En-tête des colonnes

    for nom, commentaire in dictionary.items():
        writer.writerow([nom, commentaire[0], commentaire[1], commentaire[2]])
"""
# Add par la suite pour la création d'un gros dataset pour le training du modèle
chemin_fichier = 'data.csv'

with open(chemin_fichier, 'a', newline='', encoding='utf-8') as fichier_csv:
    writer = csv.writer(fichier_csv, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    for nom, commentaire in dictionary.items():
        writer.writerow([nom, commentaire[0], commentaire[1], commentaire[2]])
"""